In [7]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [8]:
# Load the Digits dataset
digits = load_digits()
digits_df = pd.DataFrame(data=np.c_[digits['data'], digits['target']],
                          columns=np.append([f'pixel_{i}' for i in range(64)], 'target'))


In [9]:
# Feature selection and feature engineering
features = [f'pixel_{i}' for i in range(64)]

In [10]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(digits_df, test_size=0.2, random_state=42)


In [11]:
# Train a VotingClassifier model
def train_voting_classifier(X_train, y_train):
    # Define individual classifiers
    clf1 = RandomForestClassifier(n_estimators=50, random_state=42)
    clf2 = GradientBoostingClassifier(n_estimators=50, random_state=42)
    clf3 = LogisticRegression(random_state=42)

    # Create a VotingClassifier
    voting_classifier = VotingClassifier(estimators=[('rf', clf1), ('gb', clf2), ('lr', clf3)], voting='soft')
    voting_classifier.fit(X_train, y_train)
    return voting_classifier

In [12]:
# Evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)
    return accuracy, report, matrix

In [13]:
# Making Predictions with test data
def make_predictions(model, X_test, features):
    predictions = model.predict(X_test)
    results_df = pd.DataFrame({'Actual': test_data['target'].values, 'Predicted': predictions})
    return results_df

In [14]:
# Preprocess the training data
X_train, y_train = train_data[features], train_data['target']

In [15]:
 # Train a VotingClassifier model
voting_classifier = train_voting_classifier(X_train, y_train)

C:\Users\rvign\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
# Evaluate the model
X_test, y_test = test_data[features], test_data['target']
accuracy, report, confusion_matrix = evaluate_model(voting_classifier, X_test, y_test)
print(f'Accuracy of the model on the test set: {accuracy:.2f}')
print('\nClassification Report:')
print(report)
print('\nConfusion Matrix:')
print(confusion_matrix)

Accuracy of the model on the test set: 0.97

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        33
         1.0       0.97      1.00      0.98        28
         2.0       1.00      1.00      1.00        33
         3.0       0.97      0.97      0.97        34
         4.0       1.00      0.98      0.99        46
         5.0       0.94      0.96      0.95        47
         6.0       0.97      0.97      0.97        35
         7.0       1.00      0.97      0.99        34
         8.0       0.97      0.97      0.97        30
         9.0       0.95      0.95      0.95        40

    accuracy                           0.97       360
   macro avg       0.98      0.98      0.98       360
weighted avg       0.98      0.97      0.98       360


Confusion Matrix:
[[33  0  0  0  0  0  0  0  0  0]
 [ 0 28  0  0  0  0  0  0  0  0]
 [ 0  0 33  0  0  0  0  0  0  0]
 [ 0  0  0 33  0  1  0  0  0  0]
 [ 0  1  0  0 45  0  0

In [17]:
# Make predictions with test data
results_df = make_predictions(voting_classifier, X_test, features)
print('\nTest Data Predictions:')
print(results_df)


Test Data Predictions:
     Actual  Predicted
0       6.0        6.0
1       9.0        9.0
2       3.0        3.0
3       7.0        7.0
4       2.0        2.0
..      ...        ...
355     4.0        4.0
356     3.0        3.0
357     8.0        8.0
358     3.0        3.0
359     5.0        5.0

[360 rows x 2 columns]
